<h1>BooleanLA</h1>

<h3>Setting up the environment</h3>

<p>
We need to setup a couple of things before defining our classes. `%run 'svg.ipynb'` imports a variety of HTML svg tags which we will use to construct our diagrams. These are just strings which we can input values into using python's str.format(). Next, itertools.combinations is imported which we use to map out a cross product from a set which will then be used for evaluating relations. The `space` variable holds a padding value which will space out each node in the diagram. And lastly, when a set is provided in the Main.ipynb, it can either be a set of sets, integers, or strings. The set's nodes are then stored and assigned a key equal to the node's value. In the case of integers and strings this works just fine. If a set of sets is provided, we can't use it to key each node (can't use lists as keys for dicts in python). validate(s) rids us of this problem, and now we can go onto defining our classes.
</p>

In [13]:
%run 'svg.ipynb' # imports diagram, circle, text, line
from itertools import permutations, chain

space = 100 # spacing in between nodes
def validate(s):
    '''allows us to handle sets, integers, and strings'''
    return str(s) if isinstance(s, list) else s

<h3>The Node Class</h3>

<p>
A node is the most fundamental component in a diagram, yet it provides tons of information about the diagram and allows us to infer certain aspects of the plot as well. As svg, a node is comprised of a circle and text inside. The class itself contains the node's text/content/value, x and y positions for both the circle and text, radius of the circle (depending on content length), the node's lesser and greater nodes based on the relation, and whether it's a maximal or minimal node. It also holds node_plot which is just a dict holding information we send to str.format(). node_plot gets updated with the class's current information everytime we display the diagram (_repr_html_() gets called).
</p>

In [6]:
class Node(object):
    '''Node Class - represents a node in a given set'''
    def __init__(self, x, y, content):
        self.x = x
        self.y = y
        self.content = content
        self.radius = 10+len(str(self.content))
        self.node_plot = {}
        self.lesser_nodes = []
        self.greater_nodes = []
        self.is_minimal = False
        self.is_maximal = False
    def update_node(self):
        self.node_plot = {
            'x': self.x,
            'y': self.y,
            'text_x': self.x-(len(str(self.content))*2.6), #-(self.radius/2),
            'text_y': self.y+5, #+(self.radius/2),
            'radius': self.radius,
            'content': self.content
        }
    def __str__(self):
        self.update_node()
        return circle.format(**self.node_plot)

<h3>The Relation Class</h3>

<p>
A relation represents a connection between two nodes which can be defined explicitly or implicitly. When two nodes are bound to a relation, a line is drawn between the two nodes in the diagram. This line may be omitted in a Hasse diagram given that there is a separate node between the two nodes in the relation. Like the Node class, Relation holds a line_plot dict which tracks the position of the two nodes and connects the line between them.
</p>

In [1]:
class Relation(object):
    '''Relation Class - represents a relation between two nodes'''
    def __init__(self, a, b):
        self.a = a
        self.b = b
        self.line_plot = None
        self.show_relation = True
    def update_relation(self):
        self.line_plot = {
            'x1': self.a.x,
            'y1': self.a.y,
            'x2': self.b.x,
            'y2': self.b.y
        }
    def __str__(self):
        if self.show_relation:
            self.update_relation()
            return line.format(**self.line_plot)+str(self.a)+str(self.b)
        else:
            return str()

In [2]:
class Sheet(object):
    def __init__(self):
        self.diagrams = []
    
    def add_diagram(self, diagram):
        self.diagrams.append(diagram)
        
    def _repr_html_(self):
        diagram_svgs = []
        for i, diagram in enumerate(self.diagrams):
            diagram_svgs.append(diagram._repr_html_())
        return ''.join(diagram_svgs)

<h3>The Diagram Class</h3>

<p>
The Diagram class first and foremost contains a dict of nodes (key is the node content, value is the Node object) and a list of Relation objects. Diagram can be used to create a generic diagram (for example, a digraph), or subclassed to create something more specific (like a Hasse Diagram). Diagram contains the title, the set of nodes, an is_relation variable which may contain a function (used when an implicit relation is specified), and an rset list variable which may contain a list of relations (used when an explicit relation is specified). create_nodes() is called in __init__ in order to create and store the Node objects within the nodes list.
</p>

<h4>Formatting</h4>
<p>
Before explaining, here is an example scenario:<br>

<code>
S = ['a', 'b', 'c', 'd']
R = [ ('a', 'b'), ('b', 'c'), ('c', 'd') ]
</code>
<br>

S is our node set and R is our explicit relation set. A diagram representing these nodes and their relations could be configured/positioned in an infinite amount of ways; which one is the best? In some cases (like this one), it can be assumed that the nodes should be evenly distributed in a square shape, but this isn't so easy when dealing with many more nodes and more complex relations. So instead of defining our set of nodes that way S defines them, use the following format:<br>

<code>
F = {
    1: ['a', 'b'],
    2: ['c', 'd']
}</code><br><br>

This says, "I want nodes a and b in the leftmost column, and nodes c and d in the rightmost column". Here's what the output looks like:<br>

<!--<img src="http://i.imgur.com/HtKWYtG.png" /><br>-->
<img src="images/example1-1.png" /><br>

Another example:<br>
<code>
F = {
    1: ['a', 'b'],
    2: ['c'],
    3: ['d']
}
</code><br>

This says, "Nodes a and b are in the leftmost column, node c is in the middle, and node d is to the right." Output:<br>

<!--<img src="http://i.imgur.com/FSWG3Oq.png" /><br>-->
<img src="images/example1-2.png" /><br>
</p>

<h4>self.path_exists</h4>
<p>
self.path_exists takes two parameters, a set of nodes and a target node. This method recursively searches for target_node through the given set of nodes, and for each node, searches through its greater_nodes member list. And from that greater_nodes list the same operation is performed until target_node or a deadend is reached. In other words, this method returns true if there exists a path from a set of nodes to target_node (via each node's greater_nodes member list), and otherwise returns false. 
</p>

<h4>self.create_nodes</h4>
<p>
self.create_nodes does only one thing: iterate over nset (the set passed to the constructor), creates a Node object from each node, and stores that Node object inside Diagram's nodes list.
</p>

<h4>self.format_nodes</h4>
<p>
self.format_nodes loops over self.nset and applies the key to each Node's x member variable. 
</p>

In [8]:
class Diagram(object):
    '''Diagram Class - abstract class for creating visual
       representations of nodes and their relations'''
    def __init__(self, title, nset):
        self.nodes = {}
        self.relations = []
        self.title = text.format(title, 15)
        self.nset = nset
        self.is_relation = None
        self.rset = None
        self.create_nodes()
    def path_exists(self, node, target_node):
        if not node.greater_nodes:
            return
        areflexive_nodes = node.greater_nodes.copy()
        if node in areflexive_nodes:
            areflexive_nodes.remove(node)
        if target_node in areflexive_nodes:
            areflexive_nodes.remove(target_node)
        for greater_node in areflexive_nodes:
            if target_node in greater_node.greater_nodes:
                return True
            self.path_exists(greater_node, target_node)
        return False
    def create_nodes(self):
        '''create and store Node objects from nset'''
        for content in list(chain(*self.nset.values())):
            self.nodes[validate(content)] = Node(0, space, content)
    def create_relations(self):
        '''relate two nodes'''
        pass
    def format_nodes(self):
        '''position nodes based on nset structure'''
        for column, nodes in self.nset.items():
            for content in nodes:
                self.nodes[validate(content)].x = column*space

<h3>The PartialOrder Class</h3>

<p>
The PartialOrder Class, like the Diagram class, is abstract and is used to add functionality to a particular diagram. By subclassing PartialOrder, a find_extrema method is inherited.
</p>

<h4>self.find_extrema</h4>
<p>
In a partial order, particular nodes are either minimals or maximals. Node X is a minimal if there isn't a node that relates to it. Conversely, node X is a maximal if X doesn't relate to any other node. self.find_extrema will traverse through Diagram's list of nodes and set the Node.is_maximal, Node.is_minimal member variables appropriately.
</p>

In [9]:
class PartialOrder(object):
    '''PartialOrder Class - abstract class which provides additional
       functionality to Diagrams'''
    def find_extrema(self):
        '''find minimals and maximals in partial order'''
        for node in self.nodes.values():
            lines = self.title.count('/text') + 1
            content = validate(node.content)
            if node.is_minimal:
                self.title += text.format('{} is a minimal.'.format(content), 15 * lines)
            if node.is_maximal:
                self.title += text.format('{} is a maximal.'.format(content), 15 * lines)

<h3>The Hasse Class</h3>

<p>
As the class profile suggests, the Hasse class is a PartialOrder Diagram. This is a base class for HasseExplicit and HasseImplicit diagrams (which we'll discuss later). If you don't know what a Hasse diagram is, refer to the <a href="http://wikipedia.com/HasseDiagram">wiki</a>.
</p>

<h4>self.adjust_node</h4>
<p>
While in the process of comparing and positioning nodes, visual representations of relations become incorrect if the previously compared/positioned nodes aren't accounted for. For example, let's say we have nodes a, b, and c. Now, a relates to b, and b relates to c. For context, here's how the Hasse diagram should look like assuming we formatted each node to separate columns:

<img src="images/example1-3.png" />

Here's what it looks like without self.adjust_node'ing our nodes:

<img src="images/example1-4.png" />

Long story short, self.adjust_node recursively adjusts each node and each nodes' lesser nodes in order to compensate for Hasse precedence.
</p>

<h4>self.determine_relations</h4>
<p>
In a Hasse diagram, certain relation paths between two nodes are omitted if there is a separate node between that relation. Consider the example in self.adjust_nodes, only let's explicitly state that a relates to c. The path representing the relation between a and c will NOT be shown and here's why: a relates to b and b relates to c. There is a separate node, b, which connects to a and c. This method (name subject to change) uses the recursive method self.path_exists to check if there are relations analogue to this example. If there is, the Relation.show_relation member variable will be set to False.
</p>

<h4>self.set_precedence</h4>
<p>
This method is relatively simple and uses methods explained above. Recall that if a relates to b, a is considered lesser than b, and b is considered greater than a. So what self.set_precedence does is iterates over Diagram.relations, generally speaking inserts a into b's lesser_nodes list and b into a's greater_nodes list. After all the greater/lesser_nodes lists are updated, self.adjust_nodes is called which positions the nodes according to their precedence.
</p>

<h4>self.to_hasse</h4>
<p>
self.to_hasse is the glue method that executes all necessary functions to create the Hasse diagram. In order, what happens is relations are analyzed and created, precedence is set, certain paths get omitted, the visual format is set, the extrema are found, and the diagram is converted to a string and returned.
</p>

<h4>self._repr_html_</h4>
<p>
All this method does is return self.to_hasse() which will be displayed as a Hasse diagram in ipynb.
</p>

In [10]:
class Hasse(PartialOrder, Diagram):
    '''
    Hasse Class - creates a Hasse diagram from a given a node set
    '''
    def __init__(self, title, nset):
        super().__init__(title, nset)
    def adjust_node(self, node):
        '''adjust y position for Hasse precedence'''
        if not node.greater_nodes:
            node.is_maximal = True
        if not node.lesser_nodes:
            node.is_minimal = True
            return
        for lesser_node in node.lesser_nodes:
            if lesser_node.content == node.content:
                continue
            lesser_node.y = node.y + 50
            self.adjust_node(lesser_node)
    def determine_relations(self):
        '''determine which relations to show once created'''
        for relation in self.relations:
            if self.path_exists(relation.a, relation.b):
                relation.show_relation = False
    def set_precedence(self):
        '''position nodes to represent precedence'''
        for relation in self.relations:
            self.nodes[validate(relation.a.content)].greater_nodes.append(relation.b)
            self.nodes[validate(relation.b.content)].lesser_nodes.append(relation.a)
        for node in list(self.nodes.values())[::-1]:
            self.adjust_node(node)
    def to_hasse(self):
        '''create Hasse diagram from given sets'''
        self.create_relations()
        self.set_precedence()
        self.determine_relations()
        self.format_nodes()
        self.find_extrema()
        relations_svg = ''.join(map(str, self.relations))
        return diagram.format(relations_svg, self.title)
    def _repr_html_(self):
        '''svg representation'''
        return self.to_hasse()

<h3>The HasseImplicit Class</h3>

<p>
HasseImplicit (as well as HasseExplicit which is discussed below) are two sides of the same coin. The only difference between these two classes is the way we define the relation between nodes. In this case, we pass a method that takes two parameters and returns a boolean value based on some sort of comparison between the parameters (a is_subset function for example). The method passed will be assigned to self.is_relation and used later.
</p>

<h4>self.create_relations</h4>
<p>
In a sense, we create an explicit relation set by first creating a list of all possible sets between all given nodes (e.g. given a set S={1,2,3}, we create R={(1,2), (1,3), (2,3)}). Then we call self.is_relation on each of these sets and then append to self.relations Relation objects representing the sets that return True.
</p>

In [11]:
class HasseImplicit(Hasse):
    '''
    Hasse Implicit Class - creates a Hasse diagram from a given a node set
    
    the relations between nodes are implicity set (is_subset, is_less_than, ...)
    '''
    def __init__(self, title, nset, is_relation):
        super().__init__(title, nset)
        self.rset = []
        self.is_relation = is_relation
    def create_relations(self):
        for pair in permutations(list(chain(*self.nset.values())), 2):
            if self.is_relation(*pair):
                pair0 = validate(pair[0])
                pair1 = validate(pair[1])
                relation = Relation(self.nodes[pair0], self.nodes[pair1])
                self.relations.append(relation)

<h3>The HasseExplicit Class</h3>

<p>
HasseExplicit exerts the same bahavior as HasseImplicit minus checking for a relation. A relation set is passed to the constructor opposed to a method which is used to check for relations.
</p>

<h4>self.create_relations</h4>
<p>
This method iterates over the given relation set, creates Relation objects for each one, then appends the relations to self.relations.
</p>

In [12]:
class HasseExplicit(Hasse):
    '''
    Hasse Explicit Class - creates a Hasse diagram from a given a node set
    
    the relations between nodes are explicitly set ([(1, 2), (2, 3), ...])
    '''
    def __init__(self, title, nset, rset):
        super().__init__(title, nset)
        self.rset = rset
    def create_relations(self):
        for r in self.rset:
            relation = Relation(self.nodes[r[0]], self.nodes[r[1]])
            self.relations.append(relation)